## INDIVIDUAL PROJECT - Andres Olivera Caceres


## Web Scrapping

In [ ]:
import requests
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [ ]:
pip install beautifulsoup4

In [ ]:
url = "https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2018-01-01/DE"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
r = requests.get(url, headers=headers)

In [ ]:
#Iniyiate BS
soup = BeautifulSoup(r.content, "html.parser")
#Find right table
table = soup.find_all('table')[0]
# get the needed part
rows = table.find_all('tr')


In [ ]:
#Start empty list
row_list = list()
# Iterate into all data and append it into a list
for tr in rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    row_list.append(row)

## Data Manipulation 

In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np 


In [ ]:
df = pd.read_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Python/Individual project/IntradayContinuousEPEXSPOT_DE.csv')

In [ ]:
df.head()

In [ ]:
timeseries = df[['DateTime','Weighted_Avg']]
timeseries.info()

In [ ]:
#Check if there are inconsitencies in value
timeseries.describe()

In [ ]:
timeseries1 = timeseries

In [ ]:
#Make 'DateTime' a date type
timeseries1['DateTime'] =  pd.to_datetime(timeseries1['DateTime'], format='%Y-%m-%d')
timeseries1.info()

In [ ]:
#Drop NA 
timeseries1 = timeseries1.dropna()
timeseries1.info()

In [ ]:
#Make DateTiem index
timeseries1 = timeseries1.set_index(['DateTime'])
timeseries1.tail(5)

## Forecast Prep

In [ ]:
import joblib 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Plot data Set
timeseries1.plot(figsize=(25,10))
plt.show()

In [ ]:
# Dicky-Fuller test to see if it's stationary (Trend)
DickyF_test = adfuller(timeseries1['Weighted_Avg']) 
print(DickyF_test)

In [ ]:
##Not necessary
# Subtract long rolling average over N steps 
#timeseries1_1 = timeseries1 - timeseries1.rolling(50).mean()  
# Drop NaN values 
#timeseries1_1 = timeseries1_1.dropna()

In [ ]:
#Check if it's statinnary
# plot_acf
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))
plot_acf(timeseries1, lags=10, zero=False, ax=ax1)
 # Make PACF plot 
plot_pacf(timeseries1, lags=10, zero=False, ax=ax2) 
plt.show()

## Forecast

In [ ]:
!pip install pmdarima
from pmdarima import auto_arima
from pmdarima.arima import auto_arima

In [ ]:
# Get best model using auto_arima 
best_model = auto_arima( timeseries1,             # data 
                          d=0,            # non-seasonal difference order 
                          start_p=1,      # initial guess for p                          
                          start_q=1,      # initial guess for q 
                          max_p=3,        # max value of p to test                          
                          max_q=3,        # max value of q to test
                          information_criterion='aic', 
                          trace=True,
                          error_action='ignore'
                        )

In [ ]:
##Print results of best model
# Make in-sample prediction forecast = results.get_prediction(start=-25
print(best_model.summary())

In [ ]:
# Plot graphs to analyze results
best_model.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
#Recheck with this procedure
order_aic_bic =[] 
# Loop over AR order 
for p in range(4):    
    # Loop over MA order   
    for q in range(4):          
        try:             
            # Fit model             
            model = SARIMAX(timeseries1, order=(p,0,q))            
            results = model.fit() 
 
            # Add order and scores to list           
            order_aic_bic.append((p, q, results.aic, results.bic))       
        except:             
            # Print AIC and BIC as None when fails  
            order_aic_bic.append((p, q, None, None))    
            
# Make DataFrame of model order and AIC/BIC scores 
order_df = pd.DataFrame(order_aic_bic, columns=['p','q', 'aic', 'bic'])

                        
print(order_df.sort_values('aic'))
                        


In [ ]:
#select model with the lowest AIC 
best_model_normal = SARIMAX(timeseries1, order=(3, 0, 1))
                        

best_model_fit = best_model_normal.fit()
print(best_model_fit.summary().tables[1])

In [ ]:
#Plot the diagnostic of the model fit
best_model_fit.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
# In sample Predictions static

# One step predictions 
forecast_pred = best_model_fit.get_prediction(start=pd.to_datetime('2018-12-01'), dynamic=False)
# forecast mean 
mean_forecast_pred= forecast_pred.predicted_mean
# Get confidence intervals of forecasts 
confidence_intervals_pred = forecast_pred.conf_int()

In [ ]:
# Plot In sample prediction
figure(figsize=(25,10)) 
# plot the amazon data
plt.plot(timeseries1['2018-11-01':], label='observed')

# plot your mean predictions
plt.plot(mean_forecast_pred['2018-11-01':], color='r', label='forecast',alpha=.5)

plt.fill_between(confidence_intervals_pred.index,confidence_intervals_pred.loc['2018-11-01':,'lower Weighted_Avg'], 
               confidence_intervals_pred.loc['2018-11-01':,'upper Weighted_Avg'], color='Pink')

# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('EXPEX Price')
plt.legend()
plt.show()

In [ ]:
# In sample prediction Dynamic

forecast_dym = best_model_fit.get_prediction(start=pd.to_datetime('2018-12-01'), dynamic=True)
# forecast mean 
mean_forecast_dym = forecast_dym.predicted_mean
# Get confidence intervals of forecasts 
confidence_intervals_dym = forecast_dym.conf_int()

In [ ]:
#Plot In sample prediction Dynamic

figure(figsize=(25,10)) 
# plot the amazon data
plt.plot(timeseries1['2018-11-01':], label='observed')

# plot your mean predictions
plt.plot(mean_forecast_dym['2018-11-01':], color='r', label='forecast',alpha=.5)

plt.fill_between(confidence_intervals_dym.index,confidence_intervals_dym.loc['2018-11-01':,'lower Weighted_Avg'], 
               confidence_intervals_dym.loc['2018-11-01':,'upper Weighted_Avg'], color='Pink')

# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('EXPEX Price')
plt.legend()
plt.show()

In [ ]:
# FORECAST

# One step predictions 
forecast_r = best_model_fit.get_forecast(steps=30)
# forecast mean 
mean_forecast_r = forecast_r.predicted_mean
# Get confidence intervals of forecasts 
confidence_intervals_r = forecast_r.conf_int()

In [ ]:
#Data prep to plot it

#Add date and make it the index
confidence_intervals_r.index = ['2019-01-01','2019-01-02', '2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07','2019-01-08','2019-01-09','2019-01-10','2019-01-11','2019-01-12','2019-01-13','2019-01-14','2019-01-15','2019-01-16','2019-01-17','2019-01-18','2019-01-19','2019-01-20','2019-01-21','2019-01-22','2019-01-23','2019-01-24','2019-01-25','2019-01-26','2019-01-27','2019-01-28','2019-01-29','2019-01-30']
confidence_intervals_r = pd.DataFrame(confidence_intervals_r)
confidence_intervals_r = confidence_intervals_r.reset_index()
confidence_intervals_r = confidence_intervals_r.rename(columns = {'index': 'DateTime'})
confidence_intervals_r['DateTime'] =  pd.to_datetime(confidence_intervals_r['DateTime'], format='%Y-%m-%d %H:%M')
confidence_intervals_r = confidence_intervals_r.set_index('DateTime')
confidence_intervals_r.head()

In [ ]:
#Data prep to plot it

#Add date and make it the index
mean_forecast_r.index = ['2019-01-01','2019-01-02', '2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07','2019-01-08','2019-01-09','2019-01-10','2019-01-11','2019-01-12','2019-01-13','2019-01-14','2019-01-15','2019-01-16','2019-01-17','2019-01-18','2019-01-19','2019-01-20','2019-01-21','2019-01-22','2019-01-23','2019-01-24','2019-01-25','2019-01-26','2019-01-27','2019-01-28','2019-01-29','2019-01-30']
mean_forecast_r = pd.DataFrame(mean_forecast_r)
mean_forecast_r = mean_forecast_r.reset_index()
mean_forecast_r = mean_forecast_r.rename(columns = {'index': 'DateTime'})
mean_forecast_r['DateTime'] =  pd.to_datetime(mean_forecast_r['DateTime'], format='%Y-%m-%d %H:%M')
mean_forecast_r = mean_forecast_r.set_index('DateTime')


In [ ]:
#Plot Forecast way 1

ax = mean_forecast_pred.plot(label='Real auction price', figsize=(14, 4))
mean_forecast_r.plot(ax=ax, label='Forecast')
ax.fill_between(confidence_intervals_r.index,
                confidence_intervals_r.iloc[:, 0],
                confidence_intervals_r.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('DateTime')
ax.set_ylabel('Prices')
plt.legend()
plt.show()

In [ ]:
# Plot Forecast way 2

figure(figsize=(25,10))
# plot the amazon data
plt.plot(mean_forecast_pred['2018-11-01':], label='observed')

# plot your mean predictions
plt.plot(mean_forecast_r, color='r', label='forecast')

plt.fill_between(confidence_intervals_r.index,confidence_intervals_r.loc['2019-01-01':,'lower Weighted_Avg'], 
               confidence_intervals_r.loc['2019-01-01':,'upper Weighted_Avg'], color='Pink')
# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('Amazon Stock Price - Close USD')
plt.legend()
plt.show()

## Visualization Bokeh

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column
from bokeh.layouts import row

# Import Panel from bokeh.models.widgets
from bokeh.models.widgets import Panel

# Import Tabs from bokeh.models.widgets
from bokeh.models.widgets import Tabs

#Import for slicer
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import Slider

from bokeh.models import HoverTool

In [ ]:
### Original Data Set ###

# Create a figure with x_axis_type='datetime': p
p5 = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars',plot_width=1200, title = 'EPEX Germany Prices 2014-2018')

# Plot date along the x-axis and price along the y-axis
p5.line(timeseries['DateTime'], timeseries['Weighted_Avg'], legend='Original Data')

p5.legend.location = 'top_left'



In [ ]:
# Data prep to plot Forecast

# Make it DataFrame and add names
a = pd.DataFrame(mean_forecast_pred, columns = ['Price'])
a = a.reset_index()
a = a.rename(columns = {'DateTime': 'DateTime'})
a.head()

In [ ]:
# Data prep to plot Forecast

#Reset index
fore = mean_forecast_r
fore = mean_forecast_r.reset_index()
fore.head()

In [ ]:
## Forecasting Graph ##

# Create a figure with x_axis_type='datetime': p
p6 = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars', plot_width=1200, title = 'One month prices forcast')

# Plot date along the x-axis and price along the y-axis
p6.line(a['DateTime'], a['Price'], legend='EXPEX Data')
                       
p6.line(fore['DateTime'],fore[0], line_color='red', legend='Forcast' )

p6.legend.location = 'top_left'
    



In [ ]:
## TABS ##

# Create tab1 from plot p1: tab1
tab1 = Panel(child=p5, title='Origianl Data')

# Create tab2 from plot p2: tab2
tab2 = Panel(child=p6, title='Forecast')

# Create tab3 from plot p3: tab3
#tab3 = Panel(child=p3, title='Asia')

# Create tab4 from plot p4: tab4
#tab4 = Panel(child=p4, title='Europe')

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2])

# Specify the name of the output_file and show the result
output_file('tabs.html')
show(layout)


In [ ]:
# Worling code but tabs are used ## LAYOUT ROW Forcasting and Initial data##

# Create a figure with x_axis_type='datetime': p
#p = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars',plot_width=500)

# Plot date along the x-axis and price along the y-axis
#p.line(timeseries['DateTime'], timeseries['Weighted_Avg'])


# Create a figure with x_axis_type='datetime': p
##p2 = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars', plot_width=500)

# Plot date along the x-axis and price along the y-axis
##p2.line(a['DateTime'], a['Price'])
                       
##p2.line(fore['DateTime'],fore[0], line_color='red' )

##rrr = row(p, p2)
                       
# Specify the name of the output file and show the result
##output_file('row.html')
##show(rrr)

In [ ]:
# Worling code but tabs are used ## LAYOUT COL Forcasting and Initial data##

# Create a figure with x_axis_type='datetime': p
##p = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars',plot_width=500)

# Plot date along the x-axis and price along the y-axis
##p.line(timeseries['DateTime'], timeseries['Weighted_Avg'])


# Create a figure with x_axis_type='datetime': p
##p2 = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='US Dollars', plot_width=1500)

# Plot date along the x-axis and price along the y-axis
##p2.line(a['DateTime'], a['Price'])
                       
##p2.line(fore['DateTime'],fore[0], line_color='red' )

##lll = row(p, p2)
                       
# Specify the name of the output file and show the result
##output_file('row.html')
##show(lll)

In [ ]:
## Slicer ##

# Create a slider: slider
#slider = Slider(title = 'my slider', start=0, end=10, step=0.1, value=2)

# Create a widgetbox layout: layout
##layout = column(widgetbox(slider), p5)

# Add the layout to the current document
##curdoc().add_root(layout)

#show(layout)

In [ ]:
# Create ColumnDataSource: source
##source = ColumnDataSource(data={'x': x, 'y': y})

# Add a line to the plot
##plot.line('x', 'y', source=source)

# Create a column layout: layout
##layout = column(widgetbox(slider), plot)

# Add the layout to the current document
##curdoc().add_root(layout)

In [ ]:
##def callback(attr, old, new):

    # Read the current value of the slider: scale
    ##scale = slider.value

    # Compute the updated y using np.sin(scale/x): new_y
    ##new_y = scale

    # Update source with the new data values
    ##y = 

# Attach the callback to the 'value' property of slider
##slider.on_change('value', callback)

# Create layout and add to current document
##layout = column(widgetbox(slider), plot)
##curdoc().add_root(layout)

In [ ]:
#import ipywidgets as widgets
#from IPython.display import display
#import matplotlib.pyplot as plt
#import numpy as np

#%matplotlib nbagg

In [ ]:
#x = np.linspace(0, 2, 1000)
#fig, ax = plt.subplots(1, figsize=(10, 4))
#plt.suptitle('Sine Wave')


#def update_plot(amp, phase, freq):
    
   # ax.clear()
    #y = amp * np.sin(freq * 2 * np.pi * x + phase * 2 * np.pi)
    #units = 'amp = {} $(psi)$ \nphase = {} $(s)$ \nfreq = {} $(Hz)$'
    
    #ax.plot(x, y, label=units.format(amp, phase, freq))
    #ax.set_xlim(x[0], x[-1])
    #ax.legend(loc=1)
    #ax.set_xlabel('$(s)$')
    #plt.show()


##amp = widgets.FloatSlider(min=1, max=10, value=1, description='Amp:')
##phase = widgets.FloatSlider(min=0, max=5, value=0, description='Phase:')
##freq = widgets.FloatSlider(min=1, max=10, value=1, description='Freq:')
    
##widgets.interactive(update_plot, amp=amp, phase=phase, freq=freq)